In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('final dataset.csv')

In [3]:
data = data.drop_duplicates()

In [4]:
x = data.iloc[:,:-1].values
y = data.iloc[:,-1].values

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[:,5] = le.fit_transform(x[:,5])
x[:,7] = le.fit_transform(x[:,7])
x[:,12] = le.fit_transform(x[:,12])
x[:,13] = le.fit_transform(x[:,13])
y = le.fit_transform(y)

In [6]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
x= scaler.fit_transform(x)

/home/turjoy/anaconda3/envs/Arafat/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
     x, y, test_size = 0.2, random_state = 100) 

In [8]:
from sklearn.decomposition import PCA 
  
pca = PCA(copy=True, iterated_power= 'auto', n_components=2, random_state= 100,
  svd_solver='randomized', tol=0.5, whiten=True) 
  
x_train_pca = pca.fit_transform(x_train) 
x_test_pca = pca.transform(x_test) 
  
explained_variance = pca.explained_variance_ratio_ 
print (explained_variance)          

[0.37159928 0.24393848]


In [9]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.recurrent import SimpleRNN
from keras.layers.recurrent import LSTM 
from keras.optimizers import Adam,RMSprop
from keras.utils import np_utils
from keras.layers import Activation

Using TensorFlow backend.


In [10]:
y_train_nn = np_utils.to_categorical(y_train, 5)
y_test_nn = np_utils.to_categorical(y_test, 5)

In [11]:
x_train_nn = np.reshape(x_train_pca, (x_train_pca.shape[0], 1, x_train_pca.shape[1]))
x_test_nn = np.reshape(x_test_pca, (x_test_pca.shape[0], 1, x_test_pca.shape[1]))

In [12]:
rnn_model = Sequential()
rnn_model.add(SimpleRNN(32, input_shape = (1,2), return_sequences=True, kernel_initializer='uniform', activation='tanh'))
rnn_model.add(Dropout(0.4))
rnn_model.add(SimpleRNN(32,activation='tanh'))
rnn_model.add(Dropout(0.4))
rnn_model.add(Dense(units = 5,activation='softmax'))
rnn_model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999,
                 epsilon=None, decay=0., amsgrad=False), metrics=['accuracy'])
print(rnn_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 1, 32)             1120      
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 32)             0         
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 32)                2080      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 165       
Total params: 3,365
Trainable params: 3,365
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
rnn_model.fit(x_train_nn, y_train_nn, validation_data = (x_test_nn, y_test_nn), epochs = 5, batch_size = 50)

Train on 838860 samples, validate on 209715 samples
Epoch 1/5
838860/838860 [==============================] - 116s 138us/step - loss: 0.3739 - acc: 0.8968 - val_loss: 0.2231 - val_acc: 0.8957
Epoch 2/5
838860/838860 [==============================] - 109s 129us/step - loss: 0.1879 - acc: 0.9582 - val_loss: 0.1356 - val_acc: 0.9740
Epoch 3/5
838860/838860 [==============================] - 109s 130us/step - loss: 0.1539 - acc: 0.9707 - val_loss: 0.1274 - val_acc: 0.9737
Epoch 4/5
838860/838860 [==============================] - 107s 128us/step - loss: 0.1445 - acc: 0.9720 - val_loss: 0.1243 - val_acc: 0.9740
Epoch 5/5
838860/838860 [==============================] - 107s 128us/step - loss: 0.1415 - acc: 0.9721 - val_loss: 0.1240 - val_acc: 0.9748


In [14]:
metrics = rnn_model.evaluate(x_test_nn, y_test_nn, verbose=1)
print("Metrics(Test loss & Test Accuracy): ")
print(metrics)

209715/209715 [==============================] - 13s 62us/step
Metrics(Test loss & Test Accuracy): 
[0.12397649821710328, 0.9747895954052749]


In [15]:
from sklearn.metrics import confusion_matrix
y_expect = y_test_nn
y_expect = np.argmax(y_expect, axis=1)
y_predict = rnn_model.predict(x_test_nn)
y_predict = np.argmax(y_predict, axis=1)
cm = confusion_matrix(y_expect, y_predict)
print (cm)

[[     0     48      0      0    332]
 [     0 187803      0      0     45]
 [     0    607      0      0      0]
 [     0    846      0      0    397]
 [     0   3012      0      0  16625]]


In [16]:
from sklearn.metrics import classification_report
print(classification_report(y_expect,y_predict))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       380
           1       0.98      1.00      0.99    187848
           2       0.00      0.00      0.00       607
           3       0.00      0.00      0.00      1243
           4       0.96      0.85      0.90     19637

   micro avg       0.97      0.97      0.97    209715
   macro avg       0.39      0.37      0.38    209715
weighted avg       0.96      0.97      0.97    209715



/home/turjoy/anaconda3/envs/Arafat/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [17]:
from keras.utils import plot_model
plot_model(rnn_model,show_shapes=True,rankdir='TB',to_file='pcaRnn.png')

In [18]:
y_train_lstm = np_utils.to_categorical(y_train, 5)
y_test_lstm = np_utils.to_categorical(y_test, 5)

In [19]:
x_train_lstm = np.reshape(x_train_pca, (x_train_pca.shape[0], 1, x_train_pca.shape[1]))
x_test_lstm = np.reshape(x_test_pca, (x_test_pca.shape[0], 1, x_test_pca.shape[1]))

In [20]:
data_dim = 2
timesteps = 1

model = Sequential()
model.add(LSTM(30, return_sequences=True,
               input_shape=(timesteps, data_dim)))  # returns a sequence of vectors of dimension 30
model.add(LSTM(30, return_sequences=True))  # returns a sequence of vectors of dimension 30
model.add(LSTM(30))  # return a single vector of dimension 30
model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0),
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1, 30)             3960      
_________________________________________________________________
lstm_2 (LSTM)                (None, 1, 30)             7320      
_________________________________________________________________
lstm_3 (LSTM)                (None, 30)                7320      
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 155       
Total params: 18,755
Trainable params: 18,755
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.fit(x_train_lstm, y_train_lstm,validation_data = (x_test_lstm, y_test_lstm), batch_size = 50, epochs = 5, verbose = 1)

Train on 838860 samples, validate on 209715 samples
Epoch 1/5
838860/838860 [==============================] - 322s 384us/step - loss: 0.1382 - acc: 0.9701 - val_loss: 0.1160 - val_acc: 0.9745
Epoch 2/5
838860/838860 [==============================] - 332s 396us/step - loss: 0.1170 - acc: 0.9743 - val_loss: 0.1135 - val_acc: 0.9750
Epoch 3/5
838860/838860 [==============================] - 309s 368us/step - loss: 0.1156 - acc: 0.9747 - val_loss: 0.1131 - val_acc: 0.9758
Epoch 4/5
838860/838860 [==============================] - 299s 356us/step - loss: 0.1157 - acc: 0.9756 - val_loss: 0.1125 - val_acc: 0.9769
Epoch 5/5
838860/838860 [==============================] - 296s 352us/step - loss: 0.1149 - acc: 0.9764 - val_loss: 0.1147 - val_acc: 0.9756


In [22]:
metrics = model.evaluate(x_test_lstm, y_test_lstm, verbose=1)
print("Metrics(Test loss & Test Accuracy): ")
print(metrics)

209715/209715 [==============================] - 21s 98us/step
Metrics(Test loss & Test Accuracy): 
[0.11466470518382871, 0.975643134732457]


In [23]:
y_expect_ls = y_test_lstm
y_expect_ls = np.argmax(y_expect_ls, axis=1)
y_predict_ls = model.predict(x_test_lstm)
y_predict_ls = np.argmax(y_predict_ls, axis=1)
cm = confusion_matrix(y_expect_ls, y_predict_ls)
print (cm)

[[     0     15     15      0    350]
 [     0 187043    599      0    206]
 [     0    289    317      0      1]
 [     2    811     25      3    402]
 [     0   2364     29      0  17244]]


In [24]:
print(classification_report(y_expect_ls,y_predict_ls))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       380
           1       0.98      1.00      0.99    187848
           2       0.32      0.52      0.40       607
           3       1.00      0.00      0.00      1243
           4       0.95      0.88      0.91     19637

   micro avg       0.98      0.98      0.98    209715
   macro avg       0.65      0.48      0.46    209715
weighted avg       0.97      0.98      0.97    209715



In [25]:
from keras.utils import plot_model
plot_model(model,show_shapes=True,rankdir='TB',to_file='PcaLstm.png')